In [1]:
# !pip install langchain

In [2]:
# !pip install Openai

In [3]:
# !pip install chromadb

In [4]:
# !pip install tiktoken

In [5]:
from langchain.vectorstores.chroma import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import DirectoryLoader, UnstructuredMarkdownLoader
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

In [6]:
########### Embeddings ###########
import os
import openai
from langchain.embeddings import OpenAIEmbeddings
 
os.environ["OPENAI_API_TYPE"] = openai.api_type = "azure"
os.environ["OPENAI_API_VERSION"] = openai.api_version = "2022-12-01" #"2023-03-15-preview"
os.environ["OPENAI_API_BASE"] = openai.api_base =  "https://idocopenaigpt.openai.azure.com/"
os.environ["OPENAI_API_KEY"] = openai.api_key = "********************************"#os.getenv("AZUREOPENAI_API_KEY")
 
embeddings = OpenAIEmbeddings(deployment="idocembedd")


In [8]:
!pip install pypdf

  Obtaining dependency information for pypdf from https://files.pythonhosted.org/packages/74/a9/5ccde1312650dd03e65350224fea85d9a430c182a01f056599cbb76f7390/pypdf-3.17.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/277.4 kB ? eta -:--:--
   - -------------------------------------- 10.2/277.4 kB ? eta -:--:--
   ----- --------------------------------- 41.0/277.4 kB 991.0 kB/s eta 0:00:01
   -------------------------------- ------- 225.3/277.4 kB 2.3 MB/s eta 0:00:01
   ---------------------------------------- 277.4/277.4 kB 2.5 MB/s eta 0:00:00


In [9]:
# Load documents

# from langchain.document_loaders import WebBaseLoader

# loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")

#loading the pdf document
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("bitcoin.pdf")
pages = loader.load()
print(len(pages))
print(pages[0].page_content[0:500])
print(pages[0].metadata)


9
Bitcoin: A Peer-to-Peer Electronic Cash System
Satoshi Nakamoto
satoshin@gmx.com
www.bitcoin.org
Abstract.  A purely peer-to-peer version of electronic cash would allow online  
payments to be sent directly from one party to another without going through a  
financial institution.  Digital signatures provide part of the solution, but the main  
benefits are lost if a trusted third party is still required to prevent double-spending.  
We propose a solution to the double-spending problem using a p
{'source': 'bitcoin.pdf', 'page': 0}


In [10]:
# loader.load()

In [11]:
# Split documents

# from langchain.text_splitter import CharacterTextSplitter

#converting documents to chunk
text_splitter = CharacterTextSplitter(chunk_size=1024, chunk_overlap=128)
chunks = text_splitter.split_documents(pages)

In [12]:
len(chunks)

9

In [14]:
# embedding model to convert chunks to embeddings

persist_directory= ".db2/"
# db = Chroma(persist_directory=persist_directory, embedding_function=embeddings)
# memory = ConversationBufferMemory(
#     memory_key="chat_history", output_key='answer', return_messages=False)




In [15]:
from langchain.chat_models import AzureChatOpenAI
qa = ConversationalRetrievalChain.from_llm(
    llm = AzureChatOpenAI(
    openai_api_base="https://openaigpt.openai.azure.com/",
    openai_api_version="2023-03-15-preview",
    deployment_name='chatllm16k',
    openai_api_key="##77*********************003fd315264",
    openai_api_type="azure",
    temperature=0
),
    chain_type="stuff",
    retriever=db.as_retriever(),
    get_chat_history=lambda o:o,
    memory=memory,
    return_generated_question=True,
    verbose=False,
)

In [16]:
response=qa({"question": "What is bitcoin", "chat_history": []})
print(response)

{'question': 'What is bitcoin', 'chat_history': '', 'answer': 'Bitcoin is a decentralized digital currency that was created in 2009 by an unknown person or group of people using the name Satoshi Nakamoto. It operates on a peer-to-peer network and allows for secure and anonymous transactions to be made without the need for a central authority, such as a bank. Bitcoin transactions are recorded on a public ledger called the blockchain, and the currency is stored in a digital wallet. Bitcoin has gained popularity as a form of investment and is also used for online purchases and money transfers.', 'generated_question': 'What is bitcoin'}


In [20]:
response=qa({"question": "What is digital signature in bitcoin", "chat_history": []})
print(response)

{'question': 'What is digital signature in bitcoin', 'chat_history': 'Human: What is bitcoin\nAI: Bitcoin is a decentralized digital currency that was created in 2009 by an unknown person or group of people using the name Satoshi Nakamoto. It operates on a peer-to-peer network and allows for secure and anonymous transactions to be made without the need for a central authority, such as a bank. Bitcoin transactions are recorded on a public ledger called the blockchain, and the currency is stored in a digital wallet. Bitcoin has gained popularity as a form of investment and is also used for online purchases and money transfers.\nHuman: What is digital signature in bitcoin\nAI: A digital signature in Bitcoin is a cryptographic technique used to verify the authenticity and integrity of a transaction. It is created using the private key of the sender and can be verified using the corresponding public key. The digital signature ensures that the transaction has not been tampered with and that 